In [1]:
import pandas as pd
import pyodbc

pd.set_option("max_rows", None)
pd.set_option('max_columns', None)

In [ ]:
#get the database

server = ## 
database = ## 
username = ## 
password = ##
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

data = pd.read_sql_query('SELECT * FROM [panorama].[dbo].[athletics]',cnxn)

data.rename(columns={"Athletics or Varsity Sports - NAME": "Label"},inplace = True)
data.rename(columns={"Country": "Type"},inplace = True)

data = data.replace(['nan'],float('NaN'))

data.head()

In [ ]:
#append logos

ids = data['Item ID'].tolist()
logos = []

for i in ids:
    logos.append("https://scoutfyimagesbucket.s3-sa-east-1.amazonaws.com/athletics/" + str(i) + ".png")

logos_data = pd.DataFrame({'Image': logos})
data = pd.concat([data, logos_data], axis = 1)
data.head()

In [ ]:
#initializing data to store/load


name = data['Label'].tolist()

ha_lst = data['HIERARCHICAL AFFILIATION'].tolist()
hv_lst = data['HIERARCHICAL - Vinculated'].tolist()    
hr_lst = data['HIERARCHICAL - Recognized'].tolist()

#types = ["Affiliation","Vinculated","Recognized"]

In [ ]:
dic_ha = {}      #to create a json file if necessary
from_ = []    #names
to_ = []      #affiliation
type_ = []

for i in range(len(name)):
    try:
        aux_lst = ha_lst[i].split("; ")
        
        dic_ha[name[i]] = aux_lst       
        from_ += [name[i]] * len(aux_lst)
        for k in aux_lst:
            to_ += [k]
            type_ += ["Affiliation"]
            
    #treating the error for "nan" values
    except:
        pass

#type_ = ["Affiliation"] * len(to_)

In [ ]:
dic_hv = {}      #to create a json file if necessary

for i in range(len(name)):
    try:
        aux_lst = hv_lst[i].split("; ")
        
        dic_hv[name[i]] = aux_lst       
        from_ += [name[i]] * len(aux_lst)
        for k in aux_lst:
            to_ += [k]
            type_ += ["Vinculated"]
            
    #treating the error for "nan" values
    except:
        pass

In [ ]:
dic_hr = {}      #to create a json file if necessary

for i in range(len(name)):
    try:
        aux_lst = hv_lst[i].split("; ")
        
        dic_hr[name[i]] = aux_lst       
        from_ += [name[i]] * len(aux_lst)
        for k in aux_lst:
            to_ += [k]
            type_ += ["Recognized"]
            
    #treating the error for "nan" values
    except:
        pass

In [ ]:
frames = [data["Label"], data["Athletics ACRONYM"], data["Athletics NICKNAME"],
          data["Type"],data["Facebook"],data["Instagram"],
         data["Twitter"],data["LinkedIn"],data["Vimeo"],data["Flickr"],
         data["Kind"],data["State"],data["City - STATE"],
         data["HIERARCHICAL AFFILIATION"], data["Image"]]

result = pd.concat(frames,axis = 1)
result.head()

In [ ]:
#generating the dataFrame
connections_data = pd.DataFrame({'From': from_,'To': to_,'Type': type_})
connections_data

In [ ]:
#writing a new the excel file
path = r"updated-files/athletics_updated_toKumu-SQL.xlsx"
writer = pd.ExcelWriter(path, engine = 'xlsxwriter')

#generating the two necessary sheets
result.to_excel(writer, sheet_name = 'Elements',index=False)
connections_data.to_excel(writer, sheet_name = 'Connections',index=False)

writer.save()
writer.close()